## PaliGemma Fine-tuning

Pretrained Paligemma를 파인튜닝하여 딥페이크 기술로 생성된 이미지를 분류하는 모델을 생성


### 환경 설정

In [ ]:
!pip install torch
!pip install transformers
!pip install peft
!pip install trl
!pip install -U bitsandbytes
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

Huggingface 로그인  
- paligemma에 대한 read 권한 확보
- 로그인 한 계정에 대하여 파인튜닝 모델 업로드 권한 확보

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### 학습 준비

학습 데이터셋 로드

In [ ]:
from datasets import load_dataset

ds = load_dataset("JamieWithofs/Deepfake-and-real-images")

In [ ]:
ds['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 140002
})

In [ ]:
train_ds = ds['test'] # 학습 환경을 고려하여 적은 데이터셋을 학습 데이터로 선정

In [ ]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 10905
})

In [ ]:
question_make = ['Is this image made by AI?' for i in range(len(train_ds['label']))] # paligemma 학습을 위하여 question 컬럼 생성
train_ds = train_ds.add_column("question", question_make) # train_ds에 question 컬럼 추가

In [ ]:
train_ds

Dataset({
    features: ['image', 'label', 'question'],
    num_rows: 10905
})

PaliGemmaProcessor는 PaliGemma 모델과 함께 사용하는 프로세서로, 모델의 입력 데이터를 적절히 전처리하고 모델의 출력을 후처리하는 역할을 수행  




In [ ]:
from transformers import PaliGemmaProcessor
model_id = "google/paligemma-3b-pt-224"
processor = PaliGemmaProcessor.from_pretrained(model_id)

In [ ]:
import torch
device = "cuda"

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
'''
collate_fn 함수는 여러 데이터 샘플을 배치로 묶기 위한 역할.
이 함수는 데이터 로더에서 호출되어 입력 데이터를 전처리합니다.
'''
def collate_fn(examples):
    texts = ["answer " + example["question"] for example in examples]
    labels= [str(example['label']) for example in examples]
    images = [example["image"].convert("RGB") for example in examples]
    tokens = processor(text=texts, images=images, suffix=labels,
                    return_tensors="pt", padding="longest",
                    tokenize_newline_separately=False)

    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens


참고한 깃허브의 코드를 살펴보면 Paligemma를 학습하는데 사용된 데이터셋과 해당 코드에서 사용하는 VQA 데이터셋이 유사하기에,  
image encoder 부분을 파인튜닝 하지 않고 텍스트 디코더 부분만을 파인튜닝하도록 설정하였다고 되어있다.  
이에 아래와 같이 특정 파라미터의 업데이트를 False로 설정하였다.  

※ 금번 딥페이크 생성 이미지 분류 역시 데이터셋이 일반적인 얼굴 사진으로 구성된 데이터셋이므로, 동일한 구성으로 설정

In [ ]:
from transformers import PaliGemmaForConditionalGeneration
import torch

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)

# 모델의 특정 파라미터 업데이트 진행 X
for param in model.vision_tower.parameters():
    param.requires_grad = False

for param in model.multi_modal_projector.parameters():
    param.requires_grad = False


/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

QLora 적용

In [ ]:
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig

# QLora 적용 (학습을 진행하는 인프라를 고려하여 QLora 적용)

# 4 bit 양자화 적용
bnb_config = BitsAndBytesConfig(        # 4-bit quantization
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

# Lora 설정
lora_config = LoraConfig(
    r=8, # r 차원
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # Lora 적용대상
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
#trainable params: 11,298,816 || all params: 2,934,634,224 || trainable%: 0.38501616002417344


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


 TrainingArguments 클래스를 사용하여 모델 학습에 대한 다양한 하이퍼파라미터를 설정

In [ ]:
from transformers import TrainingArguments
args=TrainingArguments(
            num_train_epochs=2,
            remove_unused_columns=False,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=100,
            optim="adamw_hf",
            save_strategy="steps",
            push_to_hub=True,
            save_steps=1000,
            save_total_limit=1,
            output_dir="paligemma_deepfake_2024",
            bf16=True,
            dataloader_pin_memory=False
        )


학습 진행

In [ ]:
from transformers import Trainer

trainer = Trainer(
        model=model,
        train_dataset=train_ds ,
        data_collator=collate_fn,
        args=args
        )


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


Step,Training Loss
100,1.896600
200,0.303800
300,0.280800
400,0.266700


Step,Training Loss
100,1.896600
200,0.303800
300,0.280800
400,0.266700
500,0.237300
600,0.229300
700,0.199000
800,0.185500
900,0.157200
1000,0.168000


/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


TrainOutput(global_step=1362, training_loss=0.3267286667564717, metrics={'train_runtime': 5713.2119, 'train_samples_per_second': 3.817, 'train_steps_per_second': 0.238, 'total_flos': 8.43576117492914e+16, 'train_loss': 0.3267286667564717, 'epoch': 1.9977997799779978})

인퍼런스에 사용하기 위하여 huggingface에 모델 업로드

In [ ]:
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Donguri-b/paligemma_deepfake_2024/commit/8720529e82f69c8bb9d577078f3205f0adf051d5', commit_message='End of training', commit_description='', oid='8720529e82f69c8bb9d577078f3205f0adf051d5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# model_id = "paligemma_deepfake_2024"
# model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
# processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")